<a href="https://colab.research.google.com/github/Ajou201421102/Notebook/blob/main/multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**7. Multi-head Attention**
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### **필요 패키지 import**

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

### **데이터 전처리**

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

100%|██████████| 10/10 [00:00<00:00, 29959.31it/s]

Maximum sequence length: 20


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### **Hyperparameter 세팅 및 embedding**

In [6]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

In [7]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [8]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-1.0308e+00, -2.2698e+00, -4.7008e-01,  ...,  4.5716e-01,
          -1.0289e+00, -7.1360e-01],
         [-1.2229e+00, -8.4295e-01,  6.7000e-01,  ...,  8.0676e-02,
           1.1678e+00, -1.7502e+00],
         [ 1.1010e+00,  2.6823e-01, -5.3022e-01,  ..., -8.3909e-01,
           1.9980e-01, -1.3874e-01],
         ...,
         [ 7.2466e-01,  1.0811e+00,  1.1392e+00,  ..., -1.9504e+00,
           2.7997e-02,  1.6499e+00],
         [ 7.2466e-01,  1.0811e+00,  1.1392e+00,  ..., -1.9504e+00,
           2.7997e-02,  1.6499e+00],
         [ 7.2466e-01,  1.0811e+00,  1.1392e+00,  ..., -1.9504e+00,
           2.7997e-02,  1.6499e+00]],

        [[-1.5423e+00,  2.7244e+00, -1.5258e+00,  ...,  1.0047e+00,
           4.0716e-01,  1.9683e+00],
         [ 6.1635e-01, -8.5657e-01,  4.0952e-01,  ...,  7.4823e-01,
           4.7537e-02,  1.3202e-01],
         [ 6.7164e-01,  1.7218e+00,  7.3176e-01,  ..., -1.4014e+00,
          -1.0958e+00,  1.0032e-01],
         ...,
         [ 7.2466e-01,  1

### **Linear transformation & 여러 head로 나누기**

Multi-head attention 내에서 쓰이는 linear transformation matrix들을 정의합니다.

In [9]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [10]:
w_0 = nn.Linear(d_model, d_model)

In [11]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

In [12]:
batch_size = q.shape[0]
d_k = d_model // num_heads

q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [13]:
q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### **Scaled dot-product self-attention 구현**

각 head에서 실행되는 self-attetion 과정입니다.

In [14]:
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0561, 0.0515, 0.0602,  ..., 0.0314, 0.0314, 0.0314],
          [0.0542, 0.0493, 0.0667,  ..., 0.0448, 0.0448, 0.0448],
          [0.0560, 0.0466, 0.0498,  ..., 0.0563, 0.0563, 0.0563],
          ...,
          [0.0633, 0.0743, 0.0378,  ..., 0.0667, 0.0667, 0.0667],
          [0.0633, 0.0743, 0.0378,  ..., 0.0667, 0.0667, 0.0667],
          [0.0633, 0.0743, 0.0378,  ..., 0.0667, 0.0667, 0.0667]],

         [[0.0406, 0.0668, 0.0697,  ..., 0.0617, 0.0617, 0.0617],
          [0.0340, 0.0685, 0.0610,  ..., 0.0476, 0.0476, 0.0476],
          [0.0439, 0.0595, 0.0409,  ..., 0.0434, 0.0434, 0.0434],
          ...,
          [0.0632, 0.0346, 0.0776,  ..., 0.0560, 0.0560, 0.0560],
          [0.0632, 0.0346, 0.0776,  ..., 0.0560, 0.0560, 0.0560],
          [0.0632, 0.0346, 0.0776,  ..., 0.0560, 0.0560, 0.0560]],

         [[0.0542, 0.0576, 0.0621,  ..., 0.0561, 0.0561, 0.0561],
          [0.0776, 0.0410, 0.0297,  ..., 0.0718, 0.0718, 0.0718],
          [0.0357, 0.0563, 0.0561,  ..., 0

In [15]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### **각 head의 결과물 병합**

각 head의 결과물을 concat하고 동일 차원으로 linear transformation합니다.

In [16]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [17]:
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 2.4743e-02, -1.4083e-01, -8.9019e-03,  ..., -1.3295e-01,
          -2.1958e-01, -5.1939e-02],
         [-3.4125e-02, -1.9374e-01,  1.5574e-02,  ..., -1.2056e-01,
          -2.7329e-01, -4.4029e-02],
         [-3.7092e-03, -1.8218e-01, -4.8791e-02,  ..., -8.9728e-02,
          -2.0507e-01, -4.4638e-02],
         ...,
         [-4.4125e-03, -1.9887e-01, -2.9013e-02,  ..., -8.5663e-02,
          -2.3166e-01, -2.4689e-02],
         [-4.4125e-03, -1.9887e-01, -2.9013e-02,  ..., -8.5663e-02,
          -2.3166e-01, -2.4689e-02],
         [-4.4125e-03, -1.9887e-01, -2.9013e-02,  ..., -8.5663e-02,
          -2.3166e-01, -2.4689e-02]],

        [[ 5.2682e-02, -1.5726e-01, -2.8466e-01,  ..., -1.0008e-01,
          -6.9068e-01,  3.6610e-01],
         [ 2.6201e-02, -1.2179e-01, -2.5212e-01,  ..., -6.2683e-02,
          -6.7943e-01,  3.5675e-01],
         [ 2.4370e-02, -1.2014e-01, -2.7574e-01,  ..., -9.1736e-02,
          -6.4814e-01,  3.5690e-01],
         ...,
         [ 2.5555e-02, -1

### **전체 코드**

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈을 구현하겠습니다.

In [18]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear transformation for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    q = self.w_q(q)  # (B, L, d_model)
    k = self.w_k(k)  # (B, L, d_model)
    v = self.w_v(v)  # (B, L, d_model)

    q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

In [19]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [20]:
print(outputs)
print(outputs.shape)

tensor([[[ 0.0425, -0.0136,  0.0610,  ..., -0.0844, -0.1828, -0.0227],
         [-0.0152, -0.0007,  0.0934,  ..., -0.1279, -0.2278, -0.0071],
         [-0.0320,  0.0214,  0.0298,  ..., -0.1109, -0.1069, -0.0342],
         ...,
         [-0.0328, -0.0340,  0.0954,  ..., -0.1042, -0.1930, -0.0892],
         [-0.0328, -0.0340,  0.0954,  ..., -0.1042, -0.1930, -0.0892],
         [-0.0328, -0.0340,  0.0954,  ..., -0.1042, -0.1930, -0.0892]],

        [[-0.0957,  0.1208,  0.5480,  ..., -0.2437,  0.0196, -0.1516],
         [-0.0863,  0.1007,  0.5760,  ..., -0.2550,  0.0047, -0.1541],
         [-0.0525,  0.1035,  0.4646,  ..., -0.2358,  0.0175, -0.1794],
         ...,
         [-0.0964,  0.0777,  0.5574,  ..., -0.2514, -0.0022, -0.2327],
         [-0.0964,  0.0777,  0.5574,  ..., -0.2514, -0.0022, -0.2327],
         [-0.0964,  0.0777,  0.5574,  ..., -0.2514, -0.0022, -0.2327]],

        [[-0.1056, -0.0259,  0.1893,  ...,  0.0374,  0.0035, -0.0815],
         [-0.0462, -0.0402,  0.1039,  ...,  0